In [1]:
import pandas as pd

In [2]:
df_classifier_test = pd.read_csv('data/submissions/classifier_test.csv').fillna('')

In [3]:
df_classifier_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


## pseudo labing from classifier predictions 

In [163]:
df_classifier_test_confidence = pd.read_pickle('data/classifier_test_confidence.pickle')  

In [164]:
df_classifier_test_confidence.head()

,ImageId_ClassId,Confidence
0,004f40c73.jpg,0.016510
1,006f39c41.jpg,0.045337
2,00b7fb703.jpg,0.176907
3,00bbcd9af.jpg,0.001386
4,0108ce457.jpg,0.277736


In [165]:
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence<0.05]))
print(len(df_classifier_test_confidence[df_classifier_test_confidence.Confidence>0.8]))

1404
196


I'll add the segmentation model predictions to the train dataset that have 
- < 5% confidence on negatives and 
- \> 80% confidence on positives

in the classification model.

In [7]:
df_classifier_confident = df_classifier_test_confidence[(df_classifier_test_confidence.Confidence<0.05) | 
                                                        (df_classifier_test_confidence.Confidence>0.8)]

In [8]:
df_classifier_confident = df_classifier_confident.reset_index(drop=True)

In [9]:
df_classifier_confident.to_csv('data/confident_classification.csv', index=False)

In [10]:
df_train = pd.read_csv('data/train.csv').fillna('')

In [11]:
df_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [12]:
df_submission = pd.read_csv('data/submission.csv').fillna('')

In [13]:
df_submission.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [14]:
pseudolabel = []

for ind, row in df_classifier_confident.iterrows():
    if row['Confidence'] < 0.05 or row['Confidence'] > 0.8:
        for i in range(4):
            class_id = row.ImageId_ClassId + '_' + str(i+1)

            rle = df_submission.loc[df_submission.ImageId_ClassId == class_id].EncodedPixels.values
            if len(rle) > 0:
                rle = rle[0]
            else:
                rle = None
            pseudolabel.append({
                'ImageId_ClassId': class_id,
                'EncodedPixels': rle,
            })

In [15]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [16]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [17]:
len(df_pseudolabel_test), len(df_train)

(6400, 50272)

In [291]:
df_pseudolabel_test[df_pseudolabel_test.EncodedPixels != ''].head()

,ImageId_ClassId,EncodedPixels
10,00bbcd9af.jpg_3,72786 8 73034 26 73287 32 73541 35 73796 37 74...
19,0109b68ec.jpg_4,154349 14 154599 21 154851 27 155105 29 155360...
22,010ec96b4.jpg_3,38386 1 38389 3 38393 4 38631 24 38734 5 38871...
34,01b47d973.jpg_3,244685 3 244938 7 245193 9 245448 12 245703 13...
38,01d49cd47.jpg_3,297610 1 297612 10 297705 12 297849 42 297894 ...


In [19]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [20]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [21]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(56672, 50272, 6400)

In [22]:
df_pseudo_train.to_csv('data/train_pseudolabel.csv', index=False)

## creating pseudolabels from segmentation model (for classification)

In [235]:
df_segmentation_test_confidence = pd.read_pickle('data/segmentation_test_confidence.pickle')  

In [236]:
df_segmentation_test_confidence.head()

,ImageId_ClassId,EncodedPixels,MaxConfidence,MeanConfidence,MedianConfidence,StdConfidence
0,004f40c73.jpg_1,,0.000191,1.411692e-06,1.215099e-07,0.000007
1,004f40c73.jpg_2,,0.000124,3.824979e-07,1.535455e-08,0.000003
2,004f40c73.jpg_3,,0.014290,3.501467e-05,1.083193e-07,0.000423
3,004f40c73.jpg_4,,0.000051,5.676313e-07,9.133067e-09,0.000002
4,006f39c41.jpg_1,,0.045850,1.765962e-05,1.168544e-06,0.000466


In [237]:
df_train = pd.read_csv('data/train.csv').fillna('')

In [238]:
len(df_train), len(df_train[df_train.EncodedPixels!='']), len(df_train[df_train.EncodedPixels!=''])/len(df_train)

(50272, 7095, 0.14113224061107574)

In [239]:
len(df_segmentation_test_confidence)

7204

In [240]:
positives = len(df_segmentation_test_confidence[df_segmentation_test_confidence.MaxConfidence>0.80])
negatives = len(df_segmentation_test_confidence[df_segmentation_test_confidence.MaxConfidence<0.10])

positives, negatives, negatives+positives

(899, 5945, 6844)

In [241]:
positives/len(df_segmentation_test_confidence)

0.1247917823431427

In [242]:
df_segmentation_test = df_segmentation_test_confidence[(df_segmentation_test_confidence.MaxConfidence>0.80) | 
                                                       (df_segmentation_test_confidence.MaxConfidence<0.10)]

In [243]:
len(df_segmentation_test)

6844

In [244]:
seg_test_list = list(df_segmentation_test.ImageId_ClassId.apply(lambda x: x[:-2]))

In [245]:
from collections import Counter 
  
def removeElements(lst, k): 
    counted = Counter(lst) 
    return [el for el in lst if counted[el] >= k] 

In [246]:
seg_test_list_filtered = removeElements(seg_test_list, 4)

len(seg_test_list_filtered)

5908

In [247]:
df_segmentation_test = \
    df_segmentation_test[df_segmentation_test.ImageId_ClassId.apply(lambda x: x[:-2]).isin(seg_test_list_filtered)]

In [249]:
df_segmentation_test = df_segmentation_test.reset_index(drop=True)

In [250]:
df_segmentation_test.to_csv('data/confident_segmentation.csv', index=False)

In [251]:
len(df_segmentation_test)

5908

In [267]:
pseudolabel = []

for ind, row in df_segmentation_test.iterrows():
    class_id = row.ImageId_ClassId
    rle = df_segmentation_test.loc[df_segmentation_test.ImageId_ClassId == class_id].EncodedPixels.values

    if row['MaxConfidence'] > 0.80 and len(rle) > 0: 
        rle = rle[0]
        count+=1
    else:
        rle = ''

    pseudolabel.append({
        'ImageId_ClassId': class_id,
        'EncodedPixels': rle,
    })
        
len(pseudolabel)

5908

In [268]:
df_pseudolabel_test = pd.DataFrame(pseudolabel)

In [269]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [270]:
len(df_pseudolabel_test), len(df_train)

(5908, 50272)

In [290]:
df_pseudolabel_test[df_pseudolabel_test.EncodedPixels != ''].head()

,ImageId_ClassId,EncodedPixels
10,00bbcd9af.jpg_3,72786 8 73034 26 73287 32 73541 35 73796 37 74...
19,0109b68ec.jpg_4,154349 14 154599 21 154851 27 155105 29 155360...
22,010ec96b4.jpg_3,38386 1 38389 3 38393 4 38631 24 38734 5 38871...
34,01b47d973.jpg_3,244685 3 244938 7 245193 9 245448 12 245703 13...
38,01d49cd47.jpg_3,297610 1 297612 10 297705 12 297849 42 297894 ...


In [272]:
df_pseudo_train = df_train.append(df_pseudolabel_test, ignore_index=True)

In [273]:
df_pseudo_train.head()

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,
2,0002cc93b.jpg_3,
3,0002cc93b.jpg_4,
4,00031f466.jpg_1,


In [274]:
len(df_pseudo_train), len(df_train), len(df_pseudolabel_test)

(56180, 50272, 5908)

In [275]:
df_pseudo_train.to_csv('data/train_pseudolabel_segmentation.csv', index=False)

### copy to training folder

In [276]:
len(df_pseudolabel_test)

5908

In [277]:
from shutil import copyfile

In [278]:
df_pseudolabel_test.head()

,ImageId_ClassId,EncodedPixels
0,004f40c73.jpg_1,
1,004f40c73.jpg_2,
2,004f40c73.jpg_3,
3,004f40c73.jpg_4,
4,006f39c41.jpg_1,


In [288]:
len(os.listdir('data/test_images')), len(np.unique(seg_test_list_filtered))

(1801, 1477)

In [289]:
for fname in np.unique(seg_test_list_filtered):
    copyfile(f'data/test_images/{fname}', 
             f'data/train_pseudo_images_confident/{fname}')